<a href="https://colab.research.google.com/github/Mardoc21/Central-SSA/blob/main/Gerente_Geral_V53_(Limpeza_Visual).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
import re
import json
import shutil
import zipfile
import pandas as pd
from pathlib import Path
from datetime import datetime

# Install missing libraries if not already installed
try:
    import whisper
except ImportError:
    print("Instalando bibliotecas necessárias...")
    !pip install openai-whisper torch tqdm colorama pandas openpyxl emoji
    import whisper # Try importing again after installation

from tqdm import tqdm
import emoji
from colorama import Fore, init

init(autoreset=True)

# --- CONFIGURAÇÕES ---
USER_HOME = Path.home()
# Caminho atualizado conforme solicitação do usuário
PASTA_DOWNLOADS = Path('/content/drive/MyDrive/MAquina_esteira_d6')
PASTA_DESTINO = Path.cwd() / "Conversas_Tabuladas_V50"
MODELO_IA = "base"

# --- FUNÇÕES DE LIMPEZA AVANÇADA ---

def limpar_texto_conteudo(texto):
    if not isinstance(texto, str): return ""
    # Remove caracteres de formatação do WhatsApp (LTR/RTL marks)
    texto = texto.replace('\u200e', '').replace('\u202c', '').replace('\u202a', '')
    return texto.strip()

def limpar_nome_grupo_refinado(nome_arquivo):
    # 1. Limpeza básica do arquivo zip
    nome = re.sub(r'\s\(\d+\)$', '', nome_arquivo)
    if nome.lower().endswith('.zip'): nome = nome[:-4]

    # 2. Remove "Conversa do WhatsApp com"
    match = re.search(r' com (.*)', nome, re.IGNORECASE)
    if match: nome = match.group(1).strip()

    # 3. Remove palavra "OBRAS" (solicitação específica)
    nome = re.sub(r'\bobras\b', '', nome, flags=re.IGNORECASE)

    # 4. Remove Emojis do nome do grupo
    nome = emoji.replace_emoji(nome, replace='')

    # 5. Remove espaços duplos e traços soltos
    nome = re.sub(r'\s+', ' ', nome).strip()
    nome = nome.strip('-').strip()

    return nome

def obter_primeiro_nome(nome_completo):
    # Remove emojis do nome da pessoa
    nome_limpo = emoji.replace_emoji(nome_completo, replace='')
    # Pega só o primeiro nome (separado por espaço)
    partes = nome_limpo.split()
    if partes:
        return partes[0].title() # Capitaliza (Rafael)
    return nome_limpo

def carregar_memoria(pasta):
    arq = pasta / 'memoria_transcricoes.json'
    if arq.exists():
        try: return json.load(open(arq, encoding='utf-8'))
        except: return {}
    return {}

def salvar_memoria(pasta, dados):
    json.dump(dados, open(pasta / 'memoria_transcricoes.json', 'w', encoding='utf-8'), indent=4, ensure_ascii=False)

def parse_linha_whatsapp(linha):
    # Regex ajustado para capturar mensagens
    padrao = r'^(\d{2}/\d{2}/\d{4})\s+(\d{2}:\d{2})\s+-\s+([^:]+):\s+(.*)'
    match = re.search(padrao, linha)

    if match:
        return {
            'data': match.group(1),
            'hora': match.group(2),
            'autor': limpar_texto_conteudo(match.group(3)),
            'msg': limpar_texto_conteudo(match.group(4)),
            'raw': linha
        }
    return None

# --- MAIN --- (Refatorada para retornar DataFrame)

def main_process_whatsapp():
    print(f"{Fore.CYAN}{'='*60}")
    print(f"{Fore.WHITE}   GERENTE GERAL V50 (REFINADOR VISUAL)   ")
    print(f"{Fore.CYAN}{'='*60}")

    if not PASTA_DOWNLOADS.exists():
        print(f"{Fore.RED}[ERRO] Origem não encontrada: {PASTA_DOWNLOADS}")
        return pd.DataFrame() # Retorna DataFrame vazio em caso de erro

    PASTA_DESTINO.mkdir(exist_ok=True)

    # CORREÇÃO: Busca arquivos 'Conversa*' e valida se são ZIPs, independente da extensão
    todos_arquivos = list(PASTA_DOWNLOADS.glob("Conversa*"))
    zips = [f for f in todos_arquivos if zipfile.is_zipfile(f)]

    # Ordena por data de modificação
    zips.sort(key=lambda x: x.stat().st_mtime, reverse=True)

    print(f"Arquivos ZIP detectados: {len(zips)}")

    print(f"Carregando IA ({MODELO_IA})...")
    try:
        model = whisper.load_model(MODELO_IA)
    except Exception as e:
        print(f"{Fore.RED}Erro ao carregar IA: {e}")
        return pd.DataFrame() # Retorna DataFrame vazio em caso de erro

    dados_tabela = []

    for zip_path in tqdm(zips, unit="zip", desc="Processando"):
        # Aplica a limpeza refinada no nome do grupo (Sem "Obras", sem Emojis)
        nome_grupo_final = limpar_nome_grupo_refinado(zip_path.name)

        # Pasta temporária para extração
        pasta_temp = PASTA_DESTINO / "temp" / f"temp_{int(datetime.now().timestamp())}"

        try:
            with zipfile.ZipFile(zip_path, 'r') as zf:
                pasta_temp.mkdir(parents=True, exist_ok=True)
                zf.extractall(pasta_temp)
        except:
            continue

        txt_file = list(pasta_temp.glob("*.txt"))
        if not txt_file:
            shutil.rmtree(pasta_temp, ignore_errors=True)
            continue

        # Usa memória global para não reprocessar áudios repetidos em grupos diferentes
        memoria_path = PASTA_DESTINO / "memoria_geral.json"
        if memoria_path.exists():
            memoria = json.load(open(memoria_path, encoding='utf-8'))
        else:
            memoria = {}

        try:
            lines = open(txt_file[0], 'r', encoding='utf-8').readlines()
        except:
            try:
                lines = open(txt_file[0], 'r', encoding='latin-1').readlines()
            except:
                lines = []

        houve_transcricao = False

        for line in lines:
            line = line.strip()
            if not line: continue

            parsed = parse_linha_whatsapp(line)

            if parsed:
                # 1. Aplica limpeza no nome do autor (Primeiro Nome)
                autor_curto = obter_primeiro_nome(parsed['autor'])

                # 2. Ignora mensagens de sistema
                if "As mensagens e as chamadas são protegidas" in parsed['msg']: continue

                conteudo_final = parsed['msg']
                tipo_msg = "Texto"
                arquivo_nome = ""

                # Verifica Mídia
                match_media = re.search(r'([\w-]+\.(opus|ogg|mp3|m4a|wav|jpg|mp4)) \(arquivo anexado\)', parsed['raw'], re.IGNORECASE)

                if match_media:
                    nome_arquivo = match_media.group(1)
                    extensao = match_media.group(2).lower()
                    arquivo_nome = nome_arquivo

                    if extensao in ['opus', 'ogg', 'mp3', 'm4a', 'wav']:
                        tipo_msg = "Áudio"
                        caminho_fisico = pasta_temp / nome_arquivo

                        if nome_arquivo in memoria:
                            conteudo_final = f"[ÁUDIO]: {memoria[nome_arquivo]}"
                        elif caminho_fisico.exists():
                            try:
                                res = model.transcribe(str(caminho_fisico))
                                texto_ia = res['text'].strip()
                                memoria[nome_arquivo] = texto_ia
                                conteudo_final = f"[ÁUDIO]: {texto_ia}"
                                houve_transcricao = True
                            except Exception as e:
                                conteudo_final = f"[Erro na Transcrição: {e}]"
                        else:
                            conteudo_final = "[Áudio não baixado]"

                    elif extensao in ['jpg', 'mp4']:
                        tipo_msg = "Mídia Visual"
                        conteudo_final = f"[Arquivo: {nome_arquivo}]"

                # Adiciona à tabela com os campos limpos
                dados_tabela.append({
                    'Grupo': nome_grupo_final,
                    'Data': parsed['data'],
                    'Hora': parsed['hora'],
                    'Autor': autor_curto,
                    'Conteúdo': conteudo_final,
                    'Tipo': tipo_msg,
                    'Arquivo Original': arquivo_nome
                })

            else:
                # Continuação de mensagem (multilinha)
                if dados_tabela:
                    # Adiciona quebra de linha real para o Excel
                    dados_tabela[-1]['Conteúdo'] += f"\n{limpar_texto_conteudo(line)}"

        shutil.rmtree(pasta_temp, ignore_errors=True)

        if houve_transcricao:
            json.dump(memoria, open(memoria_path, 'w', encoding='utf-8'), indent=4, ensure_ascii=False)

    # --- PREPARAR DATAFRAME ---
    print(f"\n{Fore.GREEN}Preparando DataFrame Limpo...")

    if dados_tabela:
        df = pd.DataFrame(dados_tabela)

        # Ordenação Cronológica Reversa
        df['Data_Iso'] = pd.to_datetime(df['Data'], format='%d/%m/%Y', errors='coerce')
        df = df.sort_values(by=['Data_Iso', 'Hora'], ascending=[False, False])
        df = df.drop(columns=['Data_Iso'])

        print("DataFrame processado com sucesso.")
        return df
    else:
        print("Nenhum dado encontrado.")
        return pd.DataFrame() # Retorna um DataFrame vazio se não houver dados

if __name__ == "__main__":
    df_relatorio = main_process_whatsapp()
    print("Processamento concluído. O DataFrame está disponível como 'df_relatorio'.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Instalando bibliotecas necessárias...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 20.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 12.0 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803980 sha256=844d1ed0cf3e005261b0cd7d4e3d53210a607c1f5853b15395d13a16f1962ad3
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper
   GERENTE GERAL V50 (REFINADOR VISUAL)   
[ERRO] Origem não encontrada: /content/drive/MyDrive/MAquina_esteira_d6
Processamento concluído. O Da

In [ ]:
if 'df_relatorio' in locals() and isinstance(df_relatorio, pd.DataFrame) and not df_relatorio.empty:
    print(f"Total de mensagens processadas: {len(df_relatorio)}")
    print("\n--- Primeiras 5 linhas ---")
    display(df_relatorio.head())
    print("\n--- Estrutura do DataFrame ---")
    df_relatorio.info()
else:
    print("O DataFrame 'df_relatorio' ainda não está disponível ou está vazio.")

In [ ]:
import zipfile
import os
from pathlib import Path

PASTA_DOWNLOADS = Path('/content/drive/MyDrive/1A/Histórico backup conversas')
print(f"Diagnóstico detalhado em: {PASTA_DOWNLOADS}\n")

if PASTA_DOWNLOADS.exists():
    arquivos = list(PASTA_DOWNLOADS.iterdir())

    for arq in arquivos:
        if arq.is_dir(): continue

        print(f"Analisando: {arq.name}")

        # Verifica se é um zip válido, independente da extensão
        if zipfile.is_zipfile(arq):
            print(f"  [VALIDADO] É um arquivo ZIP válido.")
            try:
                with zipfile.ZipFile(arq, 'r') as zf:
                    arquivos_internos = zf.namelist()
                    txts = [f for f in arquivos_internos if f.lower().endswith('.txt')]

                    print(f"  Conteúdo ({len(arquivos_internos)} itens):")
                    for f in arquivos_internos[:5]: # Lista os 5 primeiros
                        print(f"    - {f}")
                    if len(arquivos_internos) > 5: print("    ...")

                    if txts:
                        print(f"  --> Arquivo de texto encontrado: {txts}")
                    else:
                        print(f"  [ALERTA] NENHUM arquivo .txt encontrado na raiz do zip.")
            except Exception as e:
                print(f"  [ERRO] Falha ao ler zip: {e}")
        else:
            print(f"  [X] NÃO é um arquivo ZIP.")
        print("-"*40)
else:
    print("Pasta não encontrada.")

In [ ]:
import pandas as pd
from pathlib import Path

PASTA_DESTINO = Path.cwd() / "Conversas_Tabuladas_V50"
csv_path = PASTA_DESTINO / "Relatorio_Refinado_V50.csv"

try:
    df_relatorio = pd.read_csv(csv_path, sep=';', encoding='utf-8-sig')
    print(f"Arquivo '{csv_path.name}' carregado com sucesso!")
    display(df_relatorio.head())
except FileNotFoundError:
    print(f"[ERRO] O arquivo '{csv_path.name}' não foi encontrado em '{csv_path}'.")
except Exception as e:
    print(f"[ERRO] Ocorreu um erro ao carregar o arquivo: {e}")

In [ ]:
import os
from pathlib import Path

PASTA_DOWNLOADS = Path('/content/drive/MyDrive/VGP_SISTEMA/01_Inbox_WhatsApp')

print(f"Verificando a pasta: {PASTA_DOWNLOADS}")

if not PASTA_DOWNLOADS.exists():
    print(f"[ERRO] A pasta '{PASTA_DOWNLOADS}' N\u00c3O foi encontrada.")
elif not PASTA_DOWNLOADS.is_dir():
    print(f"[ERRO] '{PASTA_DOWNLOADS}' N\u00c3O \u00e9 um diret\u00f3rio.")
else:
    zip_files = list(PASTA_DOWNLOADS.glob("*.zip"))
    if zip_files:
        print(f"Arquivos .zip encontrados na pasta '{PASTA_DOWNLOADS}':")
        for f in zip_files:
            print(f"- {f.name}")
    else:
        print(f"Nenhum arquivo .zip encontrado na pasta '{PASTA_DOWNLOADS}'.")


# Task
Check the structure of the `df_relatorio` DataFrame by displaying its first few rows, checking data types, and summarizing the columns to understand the available data.

## Data Inspection

### Subtask:
Inspect the `df_relatorio` DataFrame to understand its structure, column types, and content.


**Reasoning**:
I will generate a code block to inspect the `df_relatorio` DataFrame. This block will verify if the DataFrame exists in the current session, and if so, it will print its shape, display the first 5 rows, and show the dataframe summary information, satisfying the subtask instructions.



In [ ]:
if 'df_relatorio' in locals() and isinstance(df_relatorio, pd.DataFrame):
    print(f"DataFrame Shape: {df_relatorio.shape}")
    print("\n--- First 5 rows ---")
    display(df_relatorio.head())
    print("\n--- DataFrame Info ---")
    df_relatorio.info()
else:
    print("The DataFrame 'df_relatorio' is not available. Please ensure the processing step was executed successfully.")